In [13]:
import numpy as np
import datetime
import matplotlib as plt

In [9]:
class vehicle_ice:
    def __init__(self, name, mpg_hwy, mpg_city, average_override=None):
        self.mpg_hwy = mpg_hwy
        self.mpg_city = mpg_city
        self.name = name
        if average_override != None:
            self.mpg_hwy = average_override
            self.mpg_city = average_override
        
    def get_ave_mpg(fract_hwy=.50):
        return fract_hwy*self.mpg_hwy + (1-fract_hwy)*self.mpg_city
    
    
class vehicle_ev:
    def __init__(self, name, kwh_per_mile):
        self.kwh_per_mile = kwh_per_mile
        
    
class vehicle_usage:
    def __init__(self, miles_per_month, fract_hwy):
        self.miles_per_month = miles_per_month
        self.fract_hwy = fract_hwy

        
class elect_tier:
    def __init__(self, start_time, end_time, dollars_per_kwh):
        self.start = start_time
        self.end_time = end_time
        self.dollars_per_kwh = dollars_per_kwh
        
        
class elect_rates:
    def __init__(self):
        self.tiers = list()
    
    def add_tier(self, start_time, end_time, dollars_per_kwh):
        self.tiers.append( elect_tier(start_time,end_time, dollars_per_kwh) )

In [12]:
gas_costs = np.linspace(2.00, 5.00, 10)

erates = elect_rates()
erates.add_tier(datetime.time(0), datetime.time(5), 0.17)

miles_per_month = 15000.0/12.0

## Cars
volt = vehicle_ev("Volt 2016", 0.37)
grand_vitara = vehicle_ice("grand vitara", 19, 23, average_override=21)
mazda_3 = vehicle_ice("Mazda 3 2.5l", 28, 39 )

In [15]:
plt.

NameError: name 'plt' is not defined